# Exercise 4: Match the searches with bookings

For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match.

* For the bookings file, origin and destination are the columns dep_ports and arr_port, respectively.
* Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

### Plan of action:

* Get familiar with the data
* Select the data
* Decide what to do with NaNs
* Make processing plan
* Develop code that works with a sample
* Adjust the code to work with Big data
* Test big data approach on a sample
* Run program with big data

Ya estamos familiariazdos con el set de datos por los anteriores ejercicios. Así que empezamos evaluando los datos de ambos archivos y seleccionar lo que queremos o necesitamos. Anteriormente nos dimos cuenta que la misma reserva...

In [2]:
%automagic


Automagic is ON, % prefix IS NOT needed for line magics.


In [209]:
! bzcat searches.csv.bz2 | head -100000 > searches.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = searches.csv.bz2, output file = (stdout)


In [210]:
!bzip2 -f searches.sample.csv

In [211]:
! bzcat bookings.csv.bz2 | head -100000 > bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = bookings.csv.bz2, output file = (stdout)


In [212]:
!bzip2 -f bookings.sample.csv

In [213]:
ls -l

total 1106272
-rw-rw-rw- 1 pipe11 pipe11     13415 Jan 13 13:07  01_ADSC_Exercise1.ipynb
-rw-rw-rw- 1 pipe11 pipe11    205272 Jan 13 13:23  02_ADSC_Exercise2.ipynb
-rw-rw-rw- 1 pipe11 pipe11    114027 Jan 14 10:32  03_ADSC_Exercise3.ipynb
-rw-rw-rw- 1 pipe11 pipe11    239799 Jan 14 13:00  04_ADSC_Exercise4.ipynb
-rw-rw-rw- 1 pipe11 pipe11     40571 Jan 13 13:26  05_ADSC_Exercise5.ipynb
-rw-rw-rw- 1 pipe11 pipe11    212435 Jan 13 16:01  06_intro_to_pandas_practical.ipynb
-rw-rw-rw- 1 pipe11 pipe11    301678 Jan 13 18:42 'Amadeus Challenge Inclass.ipynb'
-rw-rw-rw- 1 pipe11 pipe11  54144650 Jan 14 12:33  My_searches.csv
-rw-rw-rw- 1 pipe11 pipe11  29703471 Jan 14 12:46  My_searches.csv.bz2
-rw-rw-rw- 1 pipe11 pipe11     22278 Jan 13 20:27 'Nsearches arrival flights graphic.png'
-rw-rw-rw- 1 pipe11 pipe11 554970628 Jan 11 09:18  bookings.csv.bz2
-rw-rw-rw- 1 pipe11 pipe11   5473249 Jan 14 13:01  bookings.sample.csv.bz2
-rw-rw-rw- 1 pipe11 pipe11 483188920 Jan 13 19:11  searches.csv.bz2
-r

In [214]:
import pandas as pd

sample_searches = pd.read_csv('searches.sample.csv.bz2', sep = '^', nrows = 100000)
sample_bookings = pd.read_csv('bookings.sample.csv.bz2', sep = '^', nrows = 100000)

### Bookings file

In [60]:
pd.options.display.max_columns = None
sample_bookings.sample(10)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
5471,2013-03-19 00:00:00,1P,PK,03d69b63761c9b17c2cb907f0c00c60d,f0b7dd65692fd7230f236feff8abc6b2,9a8ef4377b8922b74a9251da5ddf1247,2013-03-05 00:00:00,250,2876,JED,JED,SA,KHI,KHI,PK,KHI,KHI,PK,JED,JED,SA,KHI,KHI,PK,JEDKHI,JEDKHI,PKSA,1,JEDKHI,DG,K,Y,2013-08-07 13:50:00,2013-08-07 20:00:59,-1,2013,3,NULL
30240,2013-01-05 00:00:00,1P,US,bac4901533aee5a342030c6b20d7798a,b6359bf0377d23d26721afdd15558166,8fb5e49c1afc8a3826febd93745a3426,2013-01-05 00:00:00,1468,2417,MSP,MSP,US,MIA,MIA,US,MIA,MIA,US,FAR,FAR,US,MIA,MIA,US,FARMIA,FARMIA,USUS,0,FARMSPMIA,NV,P,F,2013-01-08 07:00:00,2013-01-08 14:43:34,1,2013,1,NULL
21132,2013-01-12 00:00:00,1P,US,71008becb70cb9156c3a7d4b2ccefec7,c28f1ba86813ba090173cbb7827fa6a2,2ef2ad6a86cf169d8685a60d3028c732,2013-01-07 00:00:00,36560,0,SDF,SDF,US,ICN,SEL,KR,SDF,SDF,US,SDF,SDF,US,ICN,SEL,KR,ICNSDF,SDFSEL,KRUS,1,SDFORDICN,UO,K,Y,2013-02-06 08:25:00,2013-02-07 15:40:03,-2,2013,1,NULL
56678,2013-01-08 00:00:00,1S,GB,d0f49af00c5dd971468f8eaf5b3d0273,114ddb09e983981c56f18a503de1456d,d7d54008e869bd5ff5a6db37e7c947d4,2013-01-08 00:00:00,3403,0,MAN,MAN,GB,BLL,BLL,DK,MAN,MAN,GB,BLL,BLL,DK,MAN,MAN,GB,BLLMAN,BLLMAN,DKGB,1,BLLMAN,LK,K,Y,2013-02-20 17:00:00,2013-02-20 17:43:10,1,2013,1,NULL
81011,2013-03-05 00:00:00,1S,CA,cce33cb9002ee539fe8c16f38980fd48,aa2bb24f0b895256cf71bcb7f6e433c5,a3df8f707a78096d5ccf9a5f31b899ce,2013-01-05 00:00:00,507,2512,YCD,YCD,CA,YQT,YQT,CA,YQT,YQT,CA,YCD,YCD,CA,YQT,YQT,CA,YCDYQT,YCDYQT,CACA,0,YCDYVRYYZYQT,KM,S,Y,2013-03-05 07:30:00,2013-03-05 18:57:17,-1,2013,3,NULL
84298,2013-01-11 00:00:00,1A,VN,ce53d8135f9911deddc7aeeb440bb21f,a2346ee0c31ab8be4794cb987ccd3b7b,473135e71dfffb73bea3e338484a9f5c,2013-01-10 00:00:00,11474,0,HAN,HAN,VN,DPS,DPS,ID,HAN,HAN,VN,DPS,DPS,ID,SIN,SIN,SG,DPSSIN,DPSSIN,IDSG,1,DPSSIN,DB,N,Y,2013-01-19 09:15:00,2013-01-19 12:00:55,-2,2013,1,NULL
17686,2013-05-28 00:00:00,1S,IT,9222ab44126311c65b4965519830863d,55e5148af5e279d2ab5484598565bc3a,e7c79b5ec1f1ef8fe6e884985813819a,2013-05-28 00:00:00,91,611,CTA,CTA,IT,OLB,OLB,IT,OLB,OLB,IT,CTA,CTA,IT,OLB,OLB,IT,CTAOLB,CTAOLB,ITIT,0,CTAOLB,SQ,A,F,2013-09-09 17:25:00,2013-09-09 18:56:08,4,2013,5,NULL
43374,2013-01-31 00:00:00,1A,GR,cc560660352eaebad60828b4b5aaf41b,0fa17cef09ded01721e2065feef2cd43,5ebff61df1f93e5fcfa8ad211a962b21,2013-01-31 00:00:00,3018,0,ATH,ATH,GR,JTR,JTR,GR,ATH,ATH,GR,ATH,ATH,GR,JTR,JTR,GR,ATHJTR,ATHJTR,GRGR,0,ATHJTR,YK,B,Y,2013-02-05 14:40:00,2013-02-05 15:43:24,1,2013,1,NULL
18804,2013-01-23 00:00:00,1S,FR,5ca6c3d674b0ff93c07127630a4d2c1f,241d738130cb157969f7066beed7e559,6547689b80745bd654592d95dac25cf8,2012-12-07 00:00:00,5281,0,SVO,MOW,RU,CDG,PAR,FR,SVO,MOW,RU,SVO,MOW,RU,CDG,PAR,FR,CDGSVO,MOWPAR,FRRU,1,SVOCDG,KP,R,Y,2013-02-12 08:20:00,2013-02-12 09:01:05,-2,2013,1,NULL
2347,2013-03-29 00:00:00,1S,US,58d06b67516ee5f9635e76844c8aec43,c7a0ad8e6fc6358d6045afb0d461869b,68dad8a5932f10c5d9d6cc53a7e4a357,2013-03-24 00:00:00,5112,0,ASE,ASE,US,ORD,CHI,US,ASE,ASE,US,ORD,CHI,US,ASE,ASE,US,ASEORD,ASECHI,USUS,0,ORDASE,FK,W,Y,2013-06-27 18:40:00,2013-06-27 20:22:55,-1,2013,3,NULL


In [30]:
sample_bookings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 38 columns):
act_date               1000000 non-null object
source                 1000000 non-null object
pos_ctry               1000000 non-null object
pos_iata               1000000 non-null object
pos_oid                1000000 non-null object
rloc                   1000000 non-null object
cre_date               1000000 non-null object
duration               1000000 non-null int64
distance               1000000 non-null int64
dep_port               1000000 non-null object
dep_city               1000000 non-null object
dep_ctry               1000000 non-null object
arr_port               1000000 non-null object
arr_city               1000000 non-null object
arr_ctry               1000000 non-null object
lst_port               1000000 non-null object
lst_city               1000000 non-null object
lst_ctry               1000000 non-null object
brd_port               1000000 non-null obje

Si tiene duplicados habría que quitarlos.

In [31]:
sample_bookings.drop_duplicates()

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
0,2013-03-05 00:00:00,1A,DE,a68dd7ae953c8acfb187a1af2dcbe123,1a11ae49fcbf545fd2afc1a24d88d2b7,ea65900e72d71f4626378e2ebd298267,2013-02-22 00:00:00,1708,0,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHR,LON,GB,ZRH,ZRH,CH,LHRZRH,LONZRH,CHGB,1,LHRZRH,VI,T,Y,2013-03-07 08:50:00,2013-03-07 11:33:37,-1,2013,3,NULL
1,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,SAL,SAL,SV,CLT,CLT,US,CLTSAL,CLTSAL,SVUS,1,SALATLCLT,NV,L,Y,2013-04-12 13:04:00,2013-04-12 22:05:40,1,2013,3,NULL
2,2013-03-26 00:00:00,1A,US,e612b9eeeee6f17f42d9b0d3b79e75ca,7437560d8f276d6d05eeb806d9e7edee,737295a86982c941f1c2da9a46a14043,2013-03-26 00:00:00,135270,0,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLT,CLT,US,SAL,SAL,SV,CLTSAL,CLTSAL,SVUS,1,CLTATLSAL,NV,U,Y,2013-07-15 07:00:00,2013-07-15 11:34:51,1,2013,3,NULL
3,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKL,AKL,NZ,SVO,MOW,RU,AKLSVO,AKLMOW,NZRU,1,AKLHKGSVO,XK,G,Y,2013-04-24 23:59:00,2013-04-25 16:06:31,1,2013,3,SYDA82546
4,2013-03-26 00:00:00,1A,AU,0f984b3bb6bd06661c95529bbd6193bc,36472c6dbaf7afec9136ac40364e2794,5ecf00fdcbcec761c43dc7285253d0c1,2013-03-26 00:00:00,30885,0,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,SVO,MOW,RU,AKL,AKL,NZ,AKLSVO,AKLMOW,NZRU,1,SVOHKGAKL,XK,G,Y,2013-05-14 20:15:00,2013-05-16 10:44:50,1,2013,3,SYDA82546
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,2013-05-18 00:00:00,1A,LK,1a5585bef21f0bf8dc865a87843635f9,6ff8fd087f94799d7aed903683384775,3b339797f7eca14c24e8b7565da822c0,2013-05-14 00:00:00,13203,0,CMB,CMB,LK,MAA,MAA,IN,CMB,CMB,LK,MAA,MAA,IN,CMB,CMB,LK,CMBMAA,CMBMAA,INLK,1,MAACMB,FV,V,Y,2013-05-24 16:05:00,2013-05-24 17:38:39,-1,2013,5,NULL
999996,2013-05-18 00:00:00,1A,LK,1a5585bef21f0bf8dc865a87843635f9,6ff8fd087f94799d7aed903683384775,3b339797f7eca14c24e8b7565da822c0,2013-05-14 00:00:00,6003,0,CMB,CMB,LK,MAA,MAA,IN,CMB,CMB,LK,MAA,MAA,IN,CMB,CMB,LK,CMBMAA,CMBMAA,INLK,1,MAACMB,FV,V,Y,2013-05-19 16:05:00,2013-05-19 17:38:39,1,2013,5,NULL
999997,2013-05-28 00:00:00,1A,US,07e98d2fd3cd4b737266b99727f7faa8,e8741eaf2fa2f71f931475d18fa72096,2b22597bdad931843c3b1d5219836e07,2013-05-28 00:00:00,328,1865,AUS,AUS,US,RDU,RDU,US,RDU,RDU,US,AUS,AUS,US,RDU,RDU,US,AUSRDU,AUSRDU,USUS,0,AUSDFWRDU,KK,Q,Y,2013-07-12 08:35:00,2013-07-12 15:03:14,1,2013,5,SEAEX38AA
999998,2013-05-28 00:00:00,1A,FR,e2679367673d50f1ee085127d8a00eba,5f26f53ca29672815e0ee3bd93a2b0e4,4eb40315a7263131ae17712cd505b62a,2013-05-28 00:00:00,588,0,TLS,TLS,FR,ORY,PAR,FR,TLS,TLS,FR,TLS,TLS,FR,ORY,PAR,FR,ORYTLS,PARTLS,FRFR,0,TLSORY,KP,S,Y,2013-06-04 10:50:00,2013-06-04 12:18:20,1,2013,5,LYSSR210Z


Pero tarda bastante y para hacerlo en todo el archivo... Vamos a tener que hacerlo por chunks

In [50]:
import pandas as pd
pd.set_option('display.max_columns', None)

bookings_sample = pd.read_csv('bookings.sample.csv.bz2', sep = '^', chunksize = 10000)
bookings_samples_dropped = []

for n, chunk in enumerate(bookings_sample):
    print(n, len(chunk))
    bookings_samples_dropped.append(chunk.drop_duplicates())
    
bookings_new = pd.concat(bookings_samples_dropped).drop_duplicates()

#Para quitar los espacios en blanco de otras columnas. Pero no me funciona bien luego.
#bookings_new = bookings_new.columns.str.strip()
#bookings_new = bookings_new.reset_index()

0 10000
1 10000
2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 9999


In [51]:
bookings_new.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
28612,2013-01-26 00:00:00,1A,PH,df9719e48f984203dee99df5f29df786,2b097aba46079ec38665864a9e2f210c,33f6079c367119e163af02025f22ab2e,2013-01-18 00:00:00,13198,0,MNL,MNL,PH,NRT,TYO,JP,MNL,MNL,PH,MNL,MNL,PH,NRT,TYO,JP,MNLNRT,MNLTYO,JPPH,1,MNLNRT,TV,V,Y,2013-04-27 09:00:00,2013-04-27 14:23:13,-2,2013,1,MNLPH21A7
10821,2013-03-28 00:00:00,1V,US,2a0249c557dee7699650a2915bfe73a2,b46b20dc57e4d60dbeca6d54c4919671,95d199a7bdeb29b8eb1ea5e27d87fd4b,2013-03-25 00:00:00,4106,0,SAN,SAN,US,BOS,BOS,US,SAN,SAN,US,SAN,SAN,US,BOS,BOS,US,BOSSAN,BOSSAN,USUS,0,SANDFWBOS,KK,M,Y,2013-04-08 12:30:00,2013-04-09 00:10:00,1,2013,3,NULL
88178,2013-03-15 00:00:00,1S,CA,a8ce9e046e00ee8545f4c8ad5e26ad1a,ddec14d12ea6687d3d28822ea0416888,6f6dcedf68249d60104259df7defd1d7,2013-03-15 00:00:00,4168,0,YUL,YMQ,CA,DCA,WAS,US,YUL,YMQ,CA,YUL,YMQ,CA,DCA,WAS,US,DCAYUL,WASYMQ,CAUS,1,YULDCA,KM,T,Y,2013-05-17 14:15:00,2013-05-17 15:58:19,2,2013,3,NULL
12857,2013-03-28 00:00:00,1A,SE,2348419a13007dbfc24c3ae11fdcc159,9068fdfb5fea275b31ef005a4ba49381,174b56e70b6f5612a8b92d953bbdfe4f,2013-03-28 00:00:00,595,0,ARN,STO,SE,MMX,MMA,SE,ARN,STO,SE,MMX,MMA,SE,ARN,STO,SE,ARNMMX,MMASTO,SESE,0,MMXARN,DU,H,Y,2013-04-04 16:00:00,2013-04-04 17:25:27,1,2013,3,NULL
44910,2013-01-23 00:00:00,1A,TR,f22e371cb4cfa55d0dad8169ca3fa93e,6dd4994a82943d8b69d0da7940e2aa52,70ce90067439cebff9cad38fcf8d5ce1,2013-01-21 00:00:00,685,6390,EZE,BUE,AR,SAP,SAP,HN,SAP,SAP,HN,EZE,BUE,AR,SAP,SAP,HN,EZESAP,BUESAP,ARHN,1,EZEPTYSAP,MW,Y,Y,2013-01-28 11:29:00,2013-01-28 19:54:22,-1,2013,1,ISTT12468


In [45]:
bookings_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99999 entries, 0 to 99998
Data columns (total 38 columns):
act_date               99999 non-null object
source                 99999 non-null object
pos_ctry               99999 non-null object
pos_iata               99999 non-null object
pos_oid                99999 non-null object
rloc                   99999 non-null object
cre_date               99999 non-null object
duration               99999 non-null int64
distance               99999 non-null int64
dep_port               99999 non-null object
dep_city               99999 non-null object
dep_ctry               99999 non-null object
arr_port               99999 non-null object
arr_city               99999 non-null object
arr_ctry               99999 non-null object
lst_port               99999 non-null object
lst_city               99999 non-null object
lst_ctry               99999 non-null object
brd_port               99999 non-null object
brd_city               99999 non-null 

### Searches file

In [48]:
sample_searches.sample(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
60723,2013-03-02,14:33:33,MPT,9aa037e8d4135205e54688c113ed7e80,DE,ATL,NYC,1,2,ATL,NYC,2013-05-26,NaN,NaN,NYC,ATL,2013-06-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,BER
15195,2013-01-16,08:29:36,MTP,236cbf458f79dbb9d5e9c9430438c3db,US,IAD,DXB,1,2,IAD,DXB,2013-03-14,NaN,NaN,DXB,IAD,2013-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN
12677,2013-01-13,21:59:45,MPT,63d35a15724f95939e6d0dd0ca39be16,DE,FRA,SPU,1,2,FRA,SPU,2013-02-28,NaN,NaN,SPU,FRA,2013-03-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,BRE
68993,2013-03-10,05:39:13,MPT,8b4fa4e121637f583bfa86ef0fa26c9e,US,EWR,MNL,1,2,EWR,MNL,2013-05-09,NaN,NaN,MNL,EWR,2013-05-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LAS
58007,2013-02-28,14:56:57,FQP,4699d6b32d0f46ba89df3d75c221def8,US,GDL,MEX,0,2,GDL,MEX,2014-01-08,KW,T,MEX,LAX,2014-01-08,KW,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,ATL


## Make a processing plan

Con los datos del archivo searches generaremos un nuevo DF de searches al que le añadiremos una columna 'Bookings' en la que pondrá 1 en el caso de que haya habido una reserva y 0 en el caso de que no haya habido reserva.

En primer lugar tenemos que limpiar los datos:

1) Quitar las cancelaciones

2) Quitar los duplicados

3) Quitar los espacios en blanco de colunm names y de cada celda del DF

4) Quitar los NaNs

5) Modelo

Quitamos las cancelaciones

In [64]:
without_cancelled.sample(5)

,act_date,source,pos_ctry,pos_iata,pos_oid,rloc,cre_date,duration,distance,dep_port,dep_city,dep_ctry,arr_port,arr_city,arr_ctry,lst_port,lst_city,lst_ctry,brd_port,brd_city,brd_ctry,off_port,off_city,off_ctry,mkt_port,mkt_city,mkt_ctry,intl,route,carrier,bkg_class,cab_class,brd_time,off_time,pax,year,month,oid
69791,2013-01-25 00:00:00,1A,US,1ba254531041b1a1888fc05d6aeaac54,9f0ed39ab39fb349b29103f593f61614,5181ea1401dc1938dd3cbf1681f150bd,2013-01-25 00:00:00,1864,0,MCO,ORL,US,DFW,DFW,US,MCO,ORL,US,DFW,DFW,US,MCO,ORL,US,DFWMCO,DFWORL,USUS,0,DFWMCO,KK,Y,Y,2013-02-01 15:25:00,2013-02-01 19:04:37,1,2013,1,NULL
81180,2013-03-24 00:00:00,1P,US,0bde7b929573cf16e0d52637cf0735d2,f187f1dfcac9833a7d7f90de67b24181,0422a98a03e8ea350f8d8137111b8237,2013-03-24 00:00:00,4814,0,OMA,OMA,US,PHX,PHX,US,OMA,OMA,US,OMA,OMA,US,PHX,PHX,US,OMAPHX,OMAPHX,USUS,0,OMAPHX,FD,G,Y,2013-04-19 05:40:00,2013-04-19 06:25:37,1,2013,3,NULL
78431,2013-01-16 00:00:00,1S,US,b87d1a15e0cdc52700628856b6199daf,1ac3c3ad5f302b5f075d9337292e6ca2,6cec2ee745f714bd8facf0cc0790d337,2013-01-16 00:00:00,5537,0,ONT,ONT,US,DFW,DFW,US,ONT,ONT,US,DFW,DFW,US,ONT,ONT,US,DFWONT,DFWONT,USUS,0,DFWONT,KK,Q,Y,2013-02-11 09:55:00,2013-02-11 10:57:37,1,2013,1,NULL
80303,2013-03-09 00:00:00,1A,FI,724f21fb091feb71e043241fe4fbc2b5,715773658985f59706880801440e0678,07a2d5551289b85bccf0720efbc7f257,2013-03-09 00:00:00,6130,0,HEL,HEL,FI,AMS,AMS,NL,HEL,HEL,FI,HEL,HEL,FI,AMS,AMS,NL,AMSHEL,AMSHEL,FINL,1,HELAMS,KJ,R,Y,2013-07-12 16:15:00,2013-07-12 17:50:19,2,2013,3,HELSG38SR
74375,2013-01-17 00:00:00,1G,NZ,8d5f2dacb37d11051f0e28ecbb57d0da,b274867bc1657e5c3ecc71c90aed17ae,25b21b7853f57a1e3b6da921e259a8d8,2013-01-11 00:00:00,40993,0,AKL,AKL,NZ,HAV,HAV,CU,AKL,AKL,NZ,SCL,SCL,CL,GUA,GUA,GT,GUASCL,GUASCL,CLGT,1,SCLPTYGUA,MW,B,Y,2013-06-08 11:26:00,2013-06-08 20:17:49,1,2013,1,NULL


In [68]:
list(without_cancelled.columns)

['act_date           ',
 'source',
 'pos_ctry',
 'pos_iata',
 'pos_oid  ',
 'rloc          ',
 'cre_date           ',
 'duration',
 'distance',
 'dep_port',
 'dep_city',
 'dep_ctry',
 'arr_port',
 'arr_city',
 'arr_ctry',
 'lst_port',
 'lst_city',
 'lst_ctry',
 'brd_port',
 'brd_city',
 'brd_ctry',
 'off_port',
 'off_city',
 'off_ctry',
 'mkt_port',
 'mkt_city',
 'mkt_ctry',
 'intl',
 'route          ',
 'carrier',
 'bkg_class',
 'cab_class',
 'brd_time           ',
 'off_time           ',
 'pax',
 'year',
 'month',
 'oid      ']

In [70]:
without_cancelled = sample_bookings[sample_bookings['pax'] > 0]
without_cancelled = without_cancelled[['cre_date           ', 'dep_port', 'arr_port']]

In [72]:
without_cancelled.sample(3)

,cre_date,dep_port,arr_port
61533,2013-03-11 00:00:00,DAC,CAN
78750,2013-01-14 00:00:00,BEG,ABV
56014,2013-05-16 00:00:00,MTY,CTU


In [76]:
without_cancelled['cre_date           '] = without_cancelled['cre_date           '].str[:10]

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
without_cancelled.sample(2)

,cre_date,dep_port,arr_port
88387,2013-03-04,GYE,SCL
12977,2013-03-28,EWR,SEA


Quitamos los duplicados y luego lo aplicamos a toda la tabla:

In [73]:
without_cancelled = without_cancelled.drop_duplicates()

In [160]:
%%time

import pandas as pd
pd.set_option('display.max_columns', None)

sample_bookings = pd.read_csv('bookings.sample.csv.bz2', sep = '^', 
                              chunksize = 10000, 
                              usecols = ['cre_date           ', 'dep_port', 'arr_port', 'pax'])
sample_bookings_clean = []

for n, chunk in enumerate(sample_bookings):
    
    print(n, len(chunk))

    chunk.columns = chunk.columns.str.strip()
    without_cancelled = chunk[chunk['pax'] > 0]
    without_cancelled['cre_date'] = without_cancelled['cre_date'].str[:10]
    chunk = without_cancelled[['cre_date', 'dep_port', 'arr_port']]
        
    sample_bookings_clean.append(chunk.drop_duplicates())
    
bookings_new = pd.concat(sample_bookings_clean).drop_duplicates()

0 10000
1 10000


/usr/lib/python3/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 9999
CPU times: user 1.53 s, sys: 46.9 ms, total: 1.58 s
Wall time: 1.57 s


In [165]:
bookings_new.sample(5)

,cre_date,dep_port,arr_port
62402,2013-02-27,MIA,LAS
83071,2013-05-15,SIN,CGK
83343,2013-05-27,MPM,GVA
72903,2013-04-03,CMH,BWI
8784,2013-03-04,PIT,FLL


In [167]:
sample_searches.sample(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice
79954,2013-03-21,21:20:44,MTC,676ba5b6a22af8f99024015d94a6bb88,GB,LON,BKK,1,2,LON,BKK,2013-04-15,NaN,NaN,BKK,LON,2013-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LON
70253,2013-03-12,21:50:35,MPT,8df3ca2bf9c54635de9cef8ae65d3da5,US,CHI,SJO,1,2,CHI,SJO,2013-05-04,NaN,NaN,SJO,CHI,2013-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HOU
65372,2013-03-07,02:51:30,MPT,d2f79dd4bfe0e15c5a14ce39891d9541,GR,NUX,LWO,1,2,NUX,LWO,2013-04-24,NaN,NaN,LWO,NUX,2013-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,ATH
24561,2013-01-25,07:46:41,FQP,b981da0514dfeafcb4a6800f7fac936a,DE,BRE,MAD,1,4,BRE,PMI,2013-02-20,KL,,PMI,MAD,2013-02-20,KL,,MAD,PMI,2013-02-22,KL,,PMI,BRE,2013-02-22,KL,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,02dd090614dd8184470db36ee19ce57a,FRA
98311,2013-04-09,13:33:46,MPT,788e52ddaa551032c69ba37cc316233f,FR,PAR,BCN,1,2,PAR,BCN,2013-04-09,NaN,NaN,BCN,PAR,2013-04-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,PAR


Vamos a hacer un merge de ambos archivos. El archivos new_bookings es bastante pequeño, ya que lo hemos limpiado y hemos seleccionado lo que queríamos.

Puedo hacer merge por chunks?

In [168]:
merged = sample_searches.merge(bookings_new,
                              left_on = ['Origin', 'Destination', 'Date'],
                              right_on = ['dep_port', 'arr_port', 'cre_date'])

In [169]:
merged

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port


In [215]:
%%time

import pandas as pd
pd.set_option('display.max_columns', None)

sample_bookings = pd.read_csv('bookings.sample.csv.bz2', sep = '^', 
                              chunksize = 10000, 
                              usecols = ['cre_date           ', 'dep_port', 'arr_port', 'pax'])
sample_bookings_clean = []

for n, chunk in enumerate(sample_bookings):
    
    print(n, len(chunk))

    chunk.columns = chunk.columns.str.strip()
    
    chunk['dep_port'] = chunk['dep_port'].str.strip()
    chunk['arr_port'] = chunk['arr_port'].str.strip()
    
    without_cancelled = chunk[chunk['pax'] > 0]
    without_cancelled['cre_date'] = without_cancelled['cre_date'].str[:10]
    without_cancelled = without_cancelled[['cre_date', 'dep_port', 'arr_port']]
        
    sample_bookings_clean.append(without_cancelled.drop_duplicates())
    
bookings_new = pd.concat(sample_bookings_clean).drop_duplicates()

0 10000
1 10000


/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 9999
CPU times: user 1.53 s, sys: 31.2 ms, total: 1.56 s
Wall time: 1.55 s


In [216]:
merged = sample_searches.merge(bookings_new,
                              left_on = ['Origin', 'Destination', 'Date'],
                              right_on = ['dep_port', 'arr_port', 'cre_date'])

In [217]:
merged.sample(5)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,cre_date,dep_port,arr_port
718,2013-02-22,20:42:31,MPT,cf4ae41b25a075c3a8fee145e1919199,NL,VIE,SOF,1,2,VIE,SOF,2013-07-30,LE,NaN,SOF,VIE,2013-08-18,LE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,d41d8cd98f00b204e9800998ecf8427e,AMS,2013-02-22,VIE,SOF
1253,2013-04-04,14:24:06,MPT,440642a9bdaeb6287f826cefd73255e8,US,SFO,RDU,0,1,SFO,RDU,2013-05-17,KD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,2013-04-04,SFO,RDU
1214,2013-03-28,06:35:12,FXX,bc762a7f5bad22cb119ca8c063f2a8b1,DE,MUC,LYS,1,2,MUC,LYS,2013-04-03,VR,J,LYS,MUC,2013-04-04,VR,J,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASI,0,0,0,f5b01d1a37bf26831071f2fe4dddefbf,FRA,2013-03-28,MUC,LYS
1220,2013-03-30,17:52:58,CCP,5c61731f33f280dfdfaecf35b6d029ce,AE,DXB,DOH,0,1,DXB,DOH,2013-04-16,JJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,AUH,2013-03-30,DXB,DOH
1087,2013-03-22,03:59:13,FFP,dfa4f720052cefaf2e1fe40befc016e6,HK,HKG,LHR,1,2,HKG,LHR,2013-03-30,MI,NaN,LHR,HKG,2013-04-06,MI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HKG,2013-03-22,HKG,LHR


In [218]:
merged['Booking'] = merged['cre_date'].notnull().astype(int)

In [237]:
merged['Booking'].sample(20)

377     1
758     1
998     1
159     1
393     1
796     1
51      1
747     1
1039    1
409     1
496     1
663     1
1001    1
938     1
467     1
852     1
886     1
652     1
1106    1
594     1
Name: Booking, dtype: int64

Nos lo pasamos a chunks para hacer el hiper merge

## Test Big Data

In [238]:
%%time

import pandas as pd
pd.set_option('display.max_columns', None)

sample_searches = pd.read_csv('searches.sample.csv.bz2', sep = '^', chunksize = 10000)

for n, chunk in enumerate(sample_searches):
    
    print(n, len(chunk))
    
    merged = chunk.merge(bookings_new,
                                   left_on = ['Origin', 'Destination', 'Date'],
                                   right_on = ['dep_port', 'arr_port', 'cre_date'],
                                   how = 'left')
    
    merged['Bookings'] = merged['cre_date'].notnull().astype(int)
    merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis = 1)
    
    merged.to_csv('My_searches.csv.bz2', compression = 'bz2', sep = '^', mode = 'a',
                 header = not bool(n))

0 10000
1 10000
2 10000
3 10000
4 10000
5 10000
6 10000
7 10000
8 10000
9 9999
CPU times: user 4.66 s, sys: 0 ns, total: 4.66 s
Wall time: 4.64 s


In [241]:
merged.sample(20)

,Date,Time,TxnCode,OfficeID,Country,Origin,Destination,RoundTrip,NbSegments,Seg1Departure,Seg1Arrival,Seg1Date,Seg1Carrier,Seg1BookingCode,Seg2Departure,Seg2Arrival,Seg2Date,Seg2Carrier,Seg2BookingCode,Seg3Departure,Seg3Arrival,Seg3Date,Seg3Carrier,Seg3BookingCode,Seg4Departure,Seg4Arrival,Seg4Date,Seg4Carrier,Seg4BookingCode,Seg5Departure,Seg5Arrival,Seg5Date,Seg5Carrier,Seg5BookingCode,Seg6Departure,Seg6Arrival,Seg6Date,Seg6Carrier,Seg6BookingCode,From,IsPublishedForNeg,IsFromInternet,IsFromVista,TerminalID,InternetOffice,Bookings
8086,2013-04-09,17:52:26,CCP,1864e5e8013d9414150e91d26b6a558b,SE,AGP,ARN,0,1,AGP,ARN,2013-06-16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,STO,0
5738,2013-04-06,21:19:32,MTP,0632d0675e69cac2473475b2261fc4b9,CA,NYC,AMS,1,2,NYC,AMS,2013-05-17,NaN,NaN,AMS,NYC,2013-05-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,YTO,0
1492,2013-04-02,20:55:20,MPT,440642a9bdaeb6287f826cefd73255e8,US,JFK,MNL,1,2,JFK,MNL,2013-04-15,MK,NaN,MNL,JFK,2013-05-13,MK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,0
5568,2013-04-06,21:10:51,MPT,1aed5e3e24c27e27a9863fa275517643,US,LAX,VVO,1,2,LAX,VVO,2013-06-01,NaN,NaN,VVO,LAX,2013-08-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,LAX,0
5245,2013-04-06,15:45:36,MPT,966af57bac7a9b07b029ebb01e0e9574,GR,KCH,PNK,0,1,KCH,PNK,2013-05-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,ATH,0
4185,2013-04-05,14:32:53,MPC,440642a9bdaeb6287f826cefd73255e8,US,AMS,DEL,1,2,AMS,DEL,2013-08-02,MK,NaN,DEL,AMS,2013-08-21,MK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,0
323,2013-04-01,22:52:28,MPT,440642a9bdaeb6287f826cefd73255e8,US,MUC,YUL,0,1,MUC,YUL,2013-07-14,FK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,HPN,0
4180,2013-04-05,12:19:11,MPT,3ac5426dd4f6bb8de28349c7b77e1ee4,IT,CPH,CAI,1,2,CPH,CAI,2013-04-12,NaN,NaN,CAI,CPH,2013-05-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,MIL,0
1679,2013-04-02,09:00:29,MPT,e917152f2688581ec4c0f40d36439a4d,NO,ROM,KUL,1,2,ROM,KUL,2013-08-06,NaN,NaN,KUL,ROM,2013-08-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,OSL,0
9439,2013-04-10,20:34:16,MPT,d03a30f66f773a65aad66e7bafd1110f,GR,MRV,MOW,1,2,MRV,MOW,2013-04-12,NaN,NaN,MOW,MRV,2013-04-14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1ASIWS,0,0,0,d41d8cd98f00b204e9800998ecf8427e,ATH,0


In [242]:
my_searches = pd.read_csv('My_searches.csv.bz2', sep = '^', nrows = 100000)

Para comprobar:

Si hago un filtro con aquellos bookings que sean mayores que 0, me saldrán todas las búsquedas que han terminado en reservas.

In [284]:
filtro_bookings = my_searches[my_searches['Bookings'] > 0]

In [287]:
my_searches.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 47 columns):
Unnamed: 0           100000 non-null int64
Date                 100000 non-null object
Time                 100000 non-null object
TxnCode              100000 non-null object
OfficeID             100000 non-null object
Country              99985 non-null object
Origin               100000 non-null object
Destination          100000 non-null object
RoundTrip            100000 non-null int64
NbSegments           100000 non-null int64
Seg1Departure        100000 non-null object
Seg1Arrival          100000 non-null object
Seg1Date             99605 non-null object
Seg1Carrier          37842 non-null object
Seg1BookingCode      10777 non-null object
Seg2Departure        72141 non-null object
Seg2Arrival          72141 non-null object
Seg2Date             71678 non-null object
Seg2Carrier          26820 non-null object
Seg2BookingCode      9272 non-null object
Seg3Departure        51

In [286]:
filtro_bookings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1276 entries, 27 to 99227
Data columns (total 47 columns):
Unnamed: 0           1276 non-null int64
Date                 1276 non-null object
Time                 1276 non-null object
TxnCode              1276 non-null object
OfficeID             1276 non-null object
Country              1276 non-null object
Origin               1276 non-null object
Destination          1276 non-null object
RoundTrip            1276 non-null int64
NbSegments           1276 non-null int64
Seg1Departure        1276 non-null object
Seg1Arrival          1276 non-null object
Seg1Date             1270 non-null object
Seg1Carrier          679 non-null object
Seg1BookingCode      293 non-null object
Seg2Departure        880 non-null object
Seg2Arrival          880 non-null object
Seg2Date             873 non-null object
Seg2Carrier          447 non-null object
Seg2BookingCode      243 non-null object
Seg3Departure        82 non-null object
Seg3Arrival          

## Run program with Big Data

Primero limpiaremos el archivo bookings por chunks.

In [302]:
%%time

import pandas as pd
pd.set_option('display.max_columns', None)

bookings_file = pd.read_csv('bookings.csv.bz2', sep = '^', 
                              chunksize = 1000000, 
                              usecols = ['cre_date           ', 'dep_port', 'arr_port', 'pax'])
bookings_clean = []

for n, chunk in enumerate(bookings_file):
    
    print(n, len(chunk))

    chunk.columns = chunk.columns.str.strip()
    
    chunk['dep_port'] = chunk['dep_port'].str.strip()
    chunk['arr_port'] = chunk['arr_port'].str.strip()
    
    without_cancelled = chunk[chunk['pax'] > 0]
    without_cancelled['cre_date'] = without_cancelled['cre_date'].str[:10]
    without_cancelled = without_cancelled[['cre_date', 'dep_port', 'arr_port']]
        
    bookings_clean.append(without_cancelled.drop_duplicates())
    
bookings_new = pd.concat(bookings_clean).drop_duplicates()

0 1000000


/usr/lib/python3/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


1 1000000
2 1000000
3 1000000
4 1000000
5 1000000
6 1000000
7 1000000
8 1000000
9 1000000
10 10
CPU times: user 2min 19s, sys: 3.59 s, total: 2min 22s
Wall time: 2min 27s


Y ahora hacemos el merge de bookings_new al archivo grande de searches.

In [304]:
%%time

import pandas as pd
pd.set_option('display.max_columns', None)

searches_file = pd.read_csv('searches.csv.bz2', sep = '^', chunksize = 1000000)

for n, chunk in enumerate(searches_file):
    
    print(n, len(chunk))
    
    merged = chunk.merge(bookings_new,
                                   left_on = ['Origin', 'Destination', 'Date'],
                                   right_on = ['dep_port', 'arr_port', 'cre_date'],
                                   how = 'left')
    
    merged['Bookings'] = merged['cre_date'].notnull().astype(int)
    merged = merged.drop(['cre_date', 'dep_port', 'arr_port'], axis = 1) 
    #Así podemos quitar estas columnas que ya nos dan igual.
    
    merged.to_csv('Searches_with_bookings.csv.bz2', compression = 'bz2', sep = '^', mode = 'a',
                 header = not bool(n))

<string>:2: DtypeWarning: Columns (44) have mixed types. Specify dtype option on import or set low_memory=False.


0 1000000


<string>:2: DtypeWarning: Columns (40,41,42,44) have mixed types. Specify dtype option on import or set low_memory=False.


1 1000000
2 1000000
3 1000000
4 1000000
5 1000000
6 1000000
7 1000000
8 1000000
9 1000000
10 1000000
11 1000000
12 1000000


<string>:2: DtypeWarning: Columns (29,30,31,32,33,34,35,36,37,38,44) have mixed types. Specify dtype option on import or set low_memory=False.


13 1000000
14 1000000
15 1000000
16 1000000
17 1000000
18 1000000
19 1000000
20 390198
CPU times: user 13min 28s, sys: 50.2 s, total: 14min 18s
Wall time: 14min 20s


In [324]:
merged['Bookings'].sample(5)

136494    0
127720    0
133576    1
22150     0
326852    0
Name: Bookings, dtype: int64

# Solución Inclass

## Match searches with bookings

- For every search in the searches file, find out whether the search ended up in a booking or not (using the info in the bookings file). For instance, search and booking origin and destination should match. 

- For the bookings file, origin and destination are the columns dep_port and arr_port, respectively. 

- Generate a CSV file with the search data, and an additional field, containing 1 if the search ended up in a booking, and 0 otherwise.

1) Get familiar with data

2) Prepare the data for processing


3) Make action plan

4) Develop the code that works with a sample


5) Adjust the code to work with Big data

6) Test big data approach on a sample

7) Run program with big data

## 2) Prepare the data for processing

### Booking

#### We didnt check for duplicates so far... What if the file is has duplicated lines?

In [ ]:
# pd.set_option('display.max_columns', None)

In [ ]:
ls

In [ ]:
import pandas as pd

In [ ]:
%%time
bi=pd.read_csv('bookings.csv.bz2', sep="^", chunksize=2000000, low_memory=False, dtype=str)
all_chunks=pd.DataFrame()

for i, chunk in enumerate(bi):
    print(i, len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    
all_chunks.to_csv('bookings_wn_dup.csv', sep='^', index=False)

In [ ]:
%%time
bi=pd.read_csv('searches.csv.bz2', sep="^", chunksize=2000000, low_memory=False, dtype=str)
all_chunks=pd.DataFrame()

for i, chunk in enumerate(bi):
    print(i, len(all_chunks))
    all_chunks=all_chunks.append(chunk)
    all_chunks.drop_duplicates(inplace=True)
    
all_chunks.to_csv('searches_wn_dup.csv', sep='^', index=False)

In [ ]:
Bookings Columns: 'act_date           ', 'dep_port', 'arr_port'
Search Columns:'Date', 'Origin', 'Destination'

#### We have seen that we have white space in some columns....

In [ ]:
b=pd.read_csv('bookings_wn_dup.csv', sep='^', usecols=['act_date           ', 'dep_port', 'arr_port'])
s=pd.read_csv('searches_wn_dup.csv', sep='^', low_memory=False)

In [ ]:
b.columns

In [ ]:
b.columns=b.columns.str.strip()

In [ ]:
b.columns

In [ ]:
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()

In [ ]:
b.arr_port[0]

In [ ]:
b.act_date[0]

Action plan:
1) quitar duplicates

2) prepare act_date in the date format of the search

3) add columns with 1s in bookings

4) merge --> left join

    Bookings Columns: 'act_date           ', 'dep_port', 'arr_port'
    Search Columns:'Date', 'Origin', 'Destination'

5) fillna(0) over the result

6) save

In [ ]:
import pandas as pd
b=pd.read_csv('bookings_wn_dup.csv', sep='^', usecols=['cre_date           ', 'dep_port', 'arr_port'])
s=pd.read_csv('searches_wn_dup.csv', sep='^', low_memory=False)

In [ ]:
b.drop_duplicates(inplace=True)
b.columns=b.columns.str.strip()
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()
b.cre_date=b.cre_date.str[0:10]
s.Date=s.Date.str[0:10]
b['Booked']=1

s_b=s.merge(b,
            how='left', 
            left_on=['Date', 'Origin', 'Destination'],
            right_on=['cre_date', 'dep_port', 'arr_port'])

In [ ]:
b.head()

In [ ]:
len(s), len(s_b)

In [ ]:
s_b.drop(['act_date', 'dep_port', 'arr_port'], axis=1, inplace=True)

In [ ]:
s_b['Booked']=s_b['Booked'].fillna(0)

In [ ]:
s_b=s_b.astype({'Booked':int})

In [ ]:
s_b[ s_b['Booked']==1 ].sample(5)

In [ ]:
import pandas as pd
b=pd.read_csv('bookings_wn_dup.csv', sep='^', usecols=['act_date           ', 'dep_port', 'arr_port'])
s=pd.read_csv('searches_wn_dup.csv', sep='^', low_memory=False)
b.drop_duplicates(inplace=True)
b.columns=b.columns.str.strip()
b.arr_port=b.arr_port.str.strip()
b.dep_port=b.dep_port.str.strip()
b.act_date=b.act_date.str[0:10]
s.Date=s.Date.str[0:10]
b['Booked']=1

s_b=s.merge(b,
            how='left', 
            left_on=['Date', 'Origin', 'Destination'],
            right_on=['act_date', 'dep_port', 'arr_port'])

s_b['Booked']=s_b['Booked'].fillna(0)
s_b.drop(['act_date', 'dep_port', 'arr_port'], axis=1, inplace=True)
s_b=s_b.astype({'Booked':int})
s_b[ s_b['Booked']==1 ].sample(5)

In [ ]:
s_b.to_csv('searches_with_bookings.csv', sep='^', index=False)

In [ ]:
! head ./searches_with_bookings.csv